In [1]:
from ultralytics import YOLO
import os
import glob
from PIL import Image
import json

# Load your trained YOLOv8 model
# The path should point to the 'best.pt' file generated after training
model = YOLO('C:/Users/CUBOX/1to100/notebooks/runs/detect/train/weights/best.pt')

# Path to your processed images and output for annotations
image_dir = 'C:/Users/CUBOX/1to100/data/processed/images'
output_json_path = 'C:/Users/CUBOX/1to100/data/processed/sample_annotations.json'

# Class names mapping (ensure this matches your model's training classes)
CLASS_NAMES = {
    0: 'header',
    1: 'passage',
    2: 'question_block',
    3: 'question_number',
    4: 'figure',
    5: 'footer'
}

all_image_annotations = []

# Get a list of all image files in the directory
image_files = glob.glob(os.path.join(image_dir, '**', '*.png'), recursive=True) + \
              glob.glob(os.path.join(image_dir, '**', '*.jpg'), recursive=True) + \
              glob.glob(os.path.join(image_dir, '**', '*.jpeg'), recursive=True)

if not image_files:
    print(f"No image files found in {image_dir}. Please ensure images are present.")
else:
    print(f"Found {len(image_files)} images for inference.")
    for image_path in image_files:
        print(f"Processing image: {image_path}")
        results = model(image_path)

        image_annotations = {
            "image_path": image_path,
            "annotations": []
        }

        for r in results:
            im_bgr = r.plot()  # plot a BGR numpy array of predictions
            im_rgb = Image.fromarray(im_bgr[..., ::-1])  # convert to RGB PIL image

            # Define output directory and filename
            output_dir = os.path.join(image_dir, "..", "inference_results") # data/processed/inference_results
            os.makedirs(output_dir, exist_ok=True)
            output_filename = os.path.basename(image_path).replace(".png", "_detected.png")
            output_filepath = os.path.join(output_dir, output_filename)

            # Save the image with bounding boxes
            im_rgb.save(output_filepath)
            print(f"Detected image saved to: {output_filepath}")

            # r.boxes.data contains [x_min, y_min, x_max, y_max, confidence, class_id]
            for *xyxy, conf, cls in r.boxes.data:
                x_min, y_min, x_max, y_max = map(float, xyxy)
                label = CLASS_NAMES.get(int(cls), "unknown")
                
                # For simplicity, text_content is empty for now. OCR would fill this.
                # Children (like question_number for question_block) are not directly handled here
                # as YOLOv8 gives flat detections. This would require post-processing.
                annotation = {
                    "label": label,
                    "bbox": [x_min, y_min, x_max, y_max],
                    "text_content": "" 
                }
                image_annotations["annotations"].append(annotation)

        all_image_annotations.append(image_annotations)

    # Save all annotations to a JSON file
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(all_image_annotations, f, ensure_ascii=False, indent=4)

    print(f"All annotations saved to {output_json_path}")


Found 20 images for inference.
Processing image: C:/Users/CUBOX/1to100/data/processed/images\2022_9월_1교시_국어영역_문제지_page_1.png

image 1/1 C:\Users\CUBOX\1to100\data\processed\images\2022_9_1___page_1.png: 640x480 1 header, 1 passage, 3 question_blocks, 3 question_numbers, 1 figure, 7 footers, 71.6ms
Speed: 3.2ms preprocess, 71.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
Detected image saved to: C:/Users/CUBOX/1to100/data/processed/images\..\inference_results\2022_9월_1교시_국어영역_문제지_page_1_detected.png
Processing image: C:/Users/CUBOX/1to100/data/processed/images\2022_9월_1교시_국어영역_문제지_page_10.png

image 1/1 C:\Users\CUBOX\1to100\data\processed\images\2022_9_1___page_10.png: 640x480 1 header, 1 passage, 1 question_block, 1 figure, 4 footers, 69.3ms
Speed: 4.5ms preprocess, 69.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
Detected image saved to: C:/Users/CUBOX/1to100/data/processed/images\..\inference_results\2022_9월_1교시_국어영역_문제지_page_10_detected